In [1]:
#spark.stop()

In [2]:
if __name__ == "__main__":

    # THIS MUST BE THE FIRST CELL IN THE NOTEBOOK
    %load_ext autoreload
    %autoreload 2
    import time
    import src.pycharm_workspace.lib_csanc109.utils.pyspark_configuration as pyspark_config

    start_time = time.time()
    sc, spark, sql_context = pyspark_config.get_spark_session(app_name="Euphoria Testing", log_level="OFF")
    print("Ended spark session: {} secs | default parallelism={}".format(time.time()-start_time, sc.defaultParallelism))

Ended spark session: 35.3096249104 secs | default parallelism=2


In [3]:
import logging
import os, sys
from datetime import datetime
import src.pycharm_workspace.lib_csanc109.utils.custom_logger as clogger

HOME_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src")

PREPAID_EUPHORIA_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "pycharm_workspace", "PrepaidRevenueBoosting")
if PREPAID_EUPHORIA_SRC not in sys.path: 
    sys.path.append(PREPAID_EUPHORIA_SRC)
    
WHOAMI= os.getenv("USER")
    
    
# Create logger object (messages to stdout and file)
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''),"logging","euphoria_"+datetime.now().strftime("%Y%m%d_%H%M%S")+".log")
logger = clogger.get_custom_logger(logging_file, std_channel=sys.stderr)
logger.info("Logging to file {}".format(logging_file))
                      
import matplotlib.pyplot as plt
%matplotlib inline  



20180713-101730 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/euphoria_20180713_101730.log


In [4]:
from dateutil.relativedelta import relativedelta
from pyspark.sql.functions import substring
from datetime import datetime
from pyspark.sql.functions import (udf,col,max as sql_max, when, count, isnull, concat, lpad, trim, lit, sum as sql_sum, length, upper)

# Set up logs.
right_now = datetime.now()
reference_date = datetime(right_now.year, right_now.month, right_now.day)
reference_date_str = reference_date.strftime('%Y%m%d')

print_string = '##### EJECUTING PREPAID EUPHORIA CAMPAIGN AT {right_now} #####'\
    .format(right_now=right_now.strftime('%Y-%m-%d %X'))
logger.info('#'*len(print_string))
logger.info(print_string)
logger.info('#'*len(print_string))

reference_month = "201805"
num_samples = 100
debug_mode = True

useful_columns_from_acFinalPrepago = ["Fecha_ejecucion", "msisdn",
                                      #"num_documento_cliente", # NIF del cliente propietario del servicio
                                      "nacionalidad", "num_prepago", "num_pospago",
                                      "age_in_years",
                                      #"tipo_documento_cliente", very uninformed
                                      "tipo_documento_comprador", # Tipo de documento (NIF, Pasaporte, Tarj. Residencia…) asociado al comprador del servicio
                                      "x_fecha_nacimiento", #"fx_activacion",
                                      "fx_1llamada", # more reliable than fx_activacion
                                      #"fx_alta_postpago"
                                      "year", "month"
                                     ]


'''
df_prepago = (spark.read.table("raw_es.customerprofilecar_prepservgsm") # se calcula a cierre de mes, en concreto el dia 2 del mes sgte
              .where((col("year") == int(reference_month[:4]))
                     & (col("month") == int(reference_month[4:]))
                    )
              .where(col("estado_servicio").isin(['CA', 'FCA', 'AC', 'FEX']))
              #.where(substring(col('fx_timestamp_servicios_prepag0'),0,7) < reference_date_str[:4]+"-"+reference_date_str[4:6])       
              .withColumn("hello", substring(col('fx_timestamp_servicios_prepag0'),0,7))
             ).select(col("msisdn_servicios_prepago").alias("msisdn"),
                      col("ius_servicios_prepago").alias("ius"),
                      col("imsi").alias("srv_imsi"),
                      col("imei").alias("srv_imei"),
                                          col("sim").alias("srv_sim"),
                                          #col("srv_last_update"),
                                          col("codigo_plan_precios").alias("srv_pricing_plan"),
                                          col("estado_servicio").alias("srv_status"),
                                          col("tac_fac").alias("srv_tac_fac"),
                                          col("fx_activacion").alias("srv_activation_ts"),
                                          col("fx_1llamada").alias("srv_firstcall_ts"),
                                          col("fx_ultimo_plan_precios").alias("srv_last_pp_change_ts"),
                                          col("contador_plan_precios").alias("srv_pp_counter"),
                     col("fx_timestamp_servicios_prepag0"), col("hello"))
'''

df_prepago = (spark.read.table("raw_es.vf_pre_ac_final") # se calcula a cierre de mes, en concreto el dia 2 del mes sgte
              .where((col("year") == int(reference_month[:4]))
                     & (col("month") == int(reference_month[4:]))
                    )
              .where(col("X_FECHA_NACIMIENTO") != "X_FE")
              .withColumn("X_FECHA_NACIMIENTO", when(length(col("X_FECHA_NACIMIENTO"))>0,col("X_FECHA_NACIMIENTO")))
              .withColumn("NUM_DOCUMENTO_CLIENTE", when(length(col("NUM_DOCUMENTO_CLIENTE"))>0,col("NUM_DOCUMENTO_CLIENTE")))
              .withColumn("NUM_DOCUMENTO_COMPRADOR", when(length(col("NUM_DOCUMENTO_COMPRADOR"))>0,col("NUM_DOCUMENTO_COMPRADOR")))
              #.withColumn("age_in_years", get_customer_age_udf(col("X_FECHA_NACIMIENTO"), reference_month))
              .withColumn("age_in_years", lit(1))
             # .withColumn("nacionalidad", substitute_crappy_characters_udf(col("nacionalidad")))
             .select(*useful_columns_from_acFinalPrepago)
.withColumnRenamed("year", "year_prepaid")
.withColumnRenamed("month", "month_prepaid"))

#################################
# Reduce the number of rows if debug mode - this will reduce the size of the dataframes due to left joins
if debug_mode: # and len(df_prepago.take(1)) > 0:
    print("Taking a sample of {}".format(num_samples))
    df_prepago = spark.createDataFrame(df_prepago.take(num_samples))





# Import PrepaidEuphoria class.
from prepaid_euphoria import PrepaidEuphoria

logger.info("THE END!!!!")


20180713-101730 [INFO ] ######################################################################
20180713-101730 [INFO ] ##### EJECUTING PREPAID EUPHORIA CAMPAIGN AT 2018-07-13 10:17:30 #####
20180713-101730 [INFO ] ######################################################################
20180713-101754 [INFO ] THE END!!!!


Taking a sample of 100


In [5]:
# df_balance_0 = (spark.read.table("raw_es.prepaid_transfbalance")             
#                               .where((col("year") == int(reference_month[:4]))
#                                    & (col("month") == int(reference_month[4:])))
#                     .select(["clave_traspaso", "msisdn_receptor", "msisdn_emisor"]))


In [6]:
df_peuph_date_master = (spark.read.table("tests_es.peuph_date_master")
                     .where((col("year") == int(reference_month[:4]))
                          & (col("month") == int(reference_month[4:]))))
                     


# Transferencias de Saldo

In [ ]:
# add_transfer_balance_information

In [19]:
df_balance_receptor = (spark.read.table("raw_es.prepaid_transfbalance")
                       .where((col("year") == int(reference_month[:4]))
                            & (col("month") == int(reference_month[4:])))
                                      .groupBy("msisdn_receptor",
                                               "year",
                                               "month",
                                               "day")
                                      .agg(sql_sum("importe_traspasado").alias("importe_traspasado_receptor"),
                                           count("importe_traspasado").alias("num_rec"))
                                      .select(col("msisdn_receptor"), 
                                              col("year"), 
                                              col("month"), 
                                              col("day"), 
                                              col("importe_traspasado_receptor"),
                                             col("num_rec"))
                       .withColumnRenamed("year", "year_receptor")
                       .withColumnRenamed("month", "month_receptor")
                       .withColumnRenamed("day", "day_receptor")
                      )
                                       

In [20]:
df_balance_emisor = (spark.read.table("raw_es.prepaid_transfbalance")
                     .where((col("year") == int(reference_month[:4]))
                          & (col("month") == int(reference_month[4:])))
                                      .groupBy("msisdn_emisor",
                                               "year",
                                               "month",
                                               "day")
                                      .agg(sql_sum("importe_traspasado").alias("importe_traspasado_emisor"),
                                           sql_sum("importe_cargo").alias("importe_cargo_emisor"),
                                           count("importe_traspasado").alias("num_em"))
                                      .select(col("msisdn_emisor"), 
                                              col("year"), 
                                              col("month"), 
                                              col("day"), 
                                              col("importe_traspasado_emisor"),
                                              col("importe_cargo_emisor"),
                                             col("num_em"))
                                            .withColumnRenamed("year", "year_emisor")
                                            .withColumnRenamed("month", "month_emisor")
                                            .withColumnRenamed("day", "day_emisor"))
                                       

In [21]:
# this dataframe has N prepago by M days 
df_prepago_cross_join = df_prepago.join(df_peuph_date_master,
                                           how="left_outer",
                                           on=((df_prepago["month_prepaid"]==df_peuph_date_master["month"]) &
                                               (df_prepago["year_prepaid"]==df_peuph_date_master["year"]))
                                           )
df_prepago_cross_join = df_prepago_cross_join.drop(*["month_prepaid", "year_prepaid"])

In [29]:
df_prepago_cross_join_2 = df_prepago_cross_join.join(df_balance_receptor,
                                           how="left",
                                           on=((df_prepago_cross_join["month"]==df_balance_receptor["month_receptor"]) &
                                               (df_prepago_cross_join["year"]==df_balance_receptor["year_receptor"]) &
                                               (df_prepago_cross_join["msisdn"]==df_balance_receptor["msisdn_receptor"]) &
                                               (df_prepago_cross_join["day"]==df_balance_receptor["day_receptor"]))
                                              )
df_prepago_cross_join_2 = df_prepago_cross_join_2.drop(*["month_receptor", "year_receptor", "day_receptor", "msisdn_receptor"])

df_prepago_cross_join_3 = df_prepago_cross_join_2.join(df_balance_emisor,
                                           how="left",
                                           on=((df_prepago_cross_join_2["month"]==df_balance_emisor["month_emisor"]) &
                                               (df_prepago_cross_join_2["year"]==df_balance_emisor["year_emisor"]) &
                                               (df_prepago_cross_join_2["msisdn"]==df_balance_emisor["msisdn_emisor"]) &
                                               (df_prepago_cross_join_2["day"]==df_balance_emisor["day_emisor"]))
                                              )
df_prepago_cross_join_3 = df_prepago_cross_join_3.drop(*["month_emisor", "year_emisor", "day_emisor", "msisdn_emisor"])


In [30]:
df_prepago_cross_join_3.columns

['Fecha_ejecucion',
 'msisdn',
 'nacionalidad',
 'num_prepago',
 'num_pospago',
 'age_in_years',
 'tipo_documento_comprador',
 'x_fecha_nacimiento',
 'fx_1llamada',
 'beginning_of_day_ts',
 'end_of_day_ts',
 'day_int',
 'day_str_1',
 'day_str_2',
 'year',
 'month',
 'day',
 'importe_traspasado_receptor',
 'num_rec',
 'importe_traspasado_emisor',
 'importe_cargo_emisor',
 'num_em']

In [12]:
# msisdn='695600350'


# import src.pycharm_workspace.lib_csanc109.utils.pyspark_utils as pyspark_utils
# from IPython.display import HTML, display
# import tabulate
# header = ["msisdn", "importe_traspasado", "num_em", "num_rec"]
# data = pyspark_utils.df_to_list(df_prepago_cross_join_3.where(col("msisdn")==msisdn).select(header))


# display(HTML(tabulate.tabulate([header ] +data, tablefmt='html', headers='firstrow')))

#df_prepago_cross_join_2.where(col("msisdn")==msisdn).show()

In [26]:
# Número de transferencias en las que el MSISDN ha sido receptor de saldo (tbal_rec_num).
# Número de transferencias en las que el MSISDN ha sido el donante de saldo (tbal_tra_num).
# Saldo recibido en transferencias (tbal_rec_amount). 
# Saldo donado en transferencias (tbal_tra_amount).

df_balance_information = (df_prepago_cross_join_3
    .withColumn("tbal_rec_amount", 1.21*col("importe_traspasado_receptor"))
    .withColumnRenamed("num_rec", "tbal_rec_num")
    .withColumn("tbal_tra_amount", 1.21*col("importe_traspasado_emisor") + 1.21*col("importe_cargo_emisor"))
    .withColumnRenamed("num_em", "tbal_tra_num"))


# TOPUPS

In [27]:
from pyspark.sql.types import IntegerType

udf_parse_importe = udf(lambda x:int(x[1:-1]),IntegerType())


df_topups = (spark.read.table("raw_es.billingtopsups_rechargescash")
                       .where((col("year") == int(reference_month[:4]))
                            & (col("month") == int(reference_month[4:])))
                       .withColumn('importe_int',udf_parse_importe('importe'))
                       .groupBy("ndc_msisdn",
                                "year",
                                "month",
                                "day")
                        .agg(sql_sum("importe_int").alias("tu_amount"),
                               count("importe_int").alias("tu_num"))
                        .select(col("ndc_msisdn"), 
                                col("year"), 
                                col("month"), 
                                col("day"), 
                                #      col("tu_ts"),
                                col("tu_amount"),
                                col("tu_num"))
                       .withColumnRenamed("year", "year_topups")
                       .withColumnRenamed("month", "month_topups")
                       .withColumnRenamed("day", "day_topups")
                      )





In [32]:
df_balance_information_2 = df_balance_information.join(df_topups,
                                           how="left",
                                           on=((df_balance_information["month"]==df_topups["month_topups"]) &
                                               (df_balance_information["year"]==df_topups["year_topups"]) &
                                               (df_balance_information["msisdn"]==df_topups["ndc_msisdn"]) &
                                               (df_balance_information["day"]==df_topups["day_topups"]))
                                              )
df_balance_information_2 = df_balance_information_2.drop(*["year_topups", "month_topups", "day_topups", "ndc_msisdn"])


# Adelantos de saldo

In [ ]:
#add_advance_balance_information

# Balance de saldo (ppio y final del dia)

# Traffic (voice and sms)

In [54]:
# voice_num - Number of calls made on a particular day,
# voice_num_distinct_rec - Number of distinct receivers,
# voice_total_duration - Total time spent making calls on a given day,
# voice_avg_duration - Average time spent on calls on a given day,
# voice_amount - Amount spend on calls on a particular day,
# sms_num - Number of SMS made on a particular day,
# sms_num_distinct_rec - Number of distinct SMS recipients on a given day.
# sms_amount - Amount spent on SMS on a particular day.
from pyspark.sql.functions import (udf,col,max as sql_max, when, countDistinct, isnull, concat, lpad, trim, lower, lit, sum as sql_avg, sum as sql_avg, length, upper)


df_call_data = (spark.read.table("raw_es.prepaid_trafficvoicesms")
                       .where((col("year") == int(reference_month[:4]))
                            & (col("month") == int(reference_month[4:])))
                 .withColumn("numeroorigen", trim(col("numeroorigen")))
                 .withColumn("numerodestino", trim(col("numerodestino")))
                 .withColumn("vozsms", lower(trim(col("vozsms"))))      
                 .withColumn("voice_amount", when(col("vozsms") == 'voz', col("importecobrado")).otherwise(0))
                 .withColumn("sms_amount", when(col("vozsms") == 'sms', col("importecobrado")).otherwise(0))
              .groupBy("numeroorigen", "year", "month", "day")
              .agg(sql_sum("voice_amount").alias("voice_amount"),
                   count("sms_amount").alias("sms_amount"),
                   sql_sum(when((col("vozsms") == 'voz'), col("airduration")).otherwise(None)).alias("voice_total_duration"),
                   sql_avg(when((col("vozsms") == 'voz'), col("airduration")).otherwise(None)).alias("voice_avg_duration"),
                   countDistinct(when((col("vozsms") == 'voz'), col("numerodestino")).otherwise(None)).alias("voice_num_distinct_rec"),
                   countDistinct(when((col("vozsms") == 'sms'), col("numerodestino")).otherwise(None)).alias("sms_num_distinct_rec")))

df_call_data.take(1)                


[Row(numeroorigen=u'653955924', year=2018, month=5, day=25, voice_amount=None, sms_amount=7, voice_total_duration=928.0, voice_avg_duration=928.0, voice_num_distinct_rec=4, sms_num_distinct_rec=0)]

# Traffic (data)

In [46]:
# data_mb - total amount of MBs consumed by data sessions.
# data_amount - total amount spent on data sessions.


# volumen data
df_data_consumed = (spark.read.table("raw_es.prepaid_trafficdata")
                       .where((col("year") == int(reference_month[:4]))
                            & (col("month") == int(reference_month[4:])))
                    .withColumn("data_mb", col("volumen")/(1024*1024))
                    .groupBy("msisdn", "dateofcall")
                    .agg(sql_sum("data_mb").alias("data_mb"),
                         sql_sum("CARGOREAL").alias("data_amount")))
                    


df_data_consumed.take(1)








[Row(msisdn=u'680007601', dateofcall=datetime.datetime(2018, 5, 25, 0, 0), voice_amount=1054.7601509094238)]

In [ ]:
        # Get voice data.
        query = """
        WITH call_data AS (
                           SELECT TRIM(numeroorigen) AS numeroorigen
                                  ,TRIM(numerodestino) AS numerodestino
                                  ,dia
                                  ,TRIM(LOWER(vozsms)) AS vozsms
                                  ,airduration
                                  ,ROW_NUMBER() OVER (PARTITION BY TRIM(numeroorigen),
                                                                   TRIM(numerodestino),
                                                                   dia,
                                                                   hora,
                                                                   TRIM(LOWER(vozsms))
                                                      ORDER BY 1) AS row_id
                           FROM raw_es.prepaid_trafficvoicesms
                           WHERE 10000*YEAR(dia) + 100*MONTH(dia) + DAY(dia) >= {min_partition_date}
                                 AND 10000*YEAR(dia) + 100*MONTH(dia) + DAY(dia) <= {max_partition_date}
                           )
             ,aggregated_call_data AS (
                                       SELECT numeroorigen
                                              ,dia
                                              ,SUM(CASE WHEN vozsms = 'voz' THEN 1 ELSE 0 END) AS voice_num
                                              ,SUM(CASE WHEN vozsms = 'sms' THEN 1 ELSE 0 END) AS sms_num
                                              ,COUNT(DISTINCT CASE WHEN vozsms = 'voz'
                                                                   THEN numerodestino
                                                                   ELSE NULL END) AS voice_num_distinct_rec
                                              ,COUNT(DISTINCT CASE WHEN vozsms = 'sms'
                                                                   THEN numerodestino
                                                                   ELSE NULL END) AS sms_num_distinct_rec
                                              ,SUM(CASE WHEN vozsms = 'voz'
                                                        THEN airduration
                                                        ELSE 0 END) AS voice_total_duration
                                              ,AVG(CASE WHEN vozsms = 'voz'
                                                        THEN airduration
                                                        ELSE NULL END) AS voice_avg_duration
                                       FROM call_data
                                       WHERE row_id = 1
                                       GROUP BY numeroorigen
                                                ,dia
                                       )
        SELECT A.*
               ,COALESCE(B.voice_num, 0) AS voice_num
               ,COALESCE(B.sms_num, 0) AS sms_num
               ,COALESCE(B.voice_num_distinct_rec, 0) AS voice_num_distinct_rec
               ,COALESCE(B.sms_num_distinct_rec, 0) AS sms_num_distinct_rec
               ,COALESCE(B.voice_total_duration, 0) AS voice_total_duration
               ,COALESCE(B.voice_avg_duration, 0) AS voice_avg_duration
        FROM {input_table_name} A
        LEFT JOIN aggregated_call_data B
        ON A.reference_ts = B.dia
           AND A.msisdn = B.numeroorigen
        """.format(input_table_name=input_table_name, min_partition_date=min_partition_date,
                   max_partition_date=max_partition_date)
        self.drop_create(query=query, output_table_name=output_table_name + '_0', advanced=True)

        query = """
        WITH call_data AS (
                           SELECT TRIM(numeroorigen) AS numeroorigen
                                  ,TRIM(numerodestino) AS numerodestino
                                  ,fechatarificacion
                                  ,TRIM(LOWER(vozsms)) AS vozsms
                                  ,importecobrado
                                  ,ROW_NUMBER() OVER (PARTITION BY TRIM(numeroorigen),
                                                                   TRIM(numerodestino),
                                                                   fechatarificacion,
                                                                   horatarificacion,
                                                                   vozsms
                                                      ORDER BY 1) AS row_id
                           FROM raw_es.prepaid_trafficvoicesms
                           WHERE 10000*YEAR(fechatarificacion)
                                 + 100*MONTH(fechatarificacion)
                                 + DAY(fechatarificacion) >= {min_partition_date}
                                 AND 10000*YEAR(fechatarificacion)
                                     + 100*MONTH(fechatarificacion)
                                     + DAY(fechatarificacion) <= {max_partition_date}
                           )
             ,aggregated_call_data AS (
                                       SELECT numeroorigen
                                              ,fechatarificacion
                                              ,SUM(CASE WHEN vozsms = 'voz'
                                                        THEN importecobrado
                                                        ELSE 0 END) AS voice_amount
                                              ,SUM(CASE WHEN vozsms = 'sms'
                                                        THEN importecobrado
                                                        ELSE 0 END) AS sms_amount
                                       FROM call_data
                                       WHERE row_id = 1
                                       GROUP BY numeroorigen, fechatarificacion
                                       )
        SELECT A.*
               ,COALESCE(B.voice_amount, 0) AS voice_amount
               ,COALESCE(B.sms_amount, 0) AS sms_amount
        FROM {output_table_name}_0 A
        LEFT JOIN aggregated_call_data B
        ON A.reference_ts = B.fechatarificacion
           AND A.msisdn = B.numeroorigen;
        DROP TABLE IF EXISTS {output_table_name}_0;
        """.format(output_table_name=output_table_name,
                   min_partition_date=min_partition_date,
                   max_partition_date=max_partition_date)
        self.drop_create(query=query, output_table_name=output_table_name, advanced=True)
        self.drop_input_table(input_table_name, condition=drop_input_table)